In [ ]:
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt
import sys
import MnistLayers

In [ ]:
def show(x,name) :
    print(x)
    print(f"{name} shape : " , x.shape)

In [ ]:
N = 2               # input 개수
n_ic = 2            # input channel
n_oc = 3            # output channel
n_ih = 5            # input height
n_iw = 4            # input weight
n_fh = n_ph = 3     # filter / pooling height
n_fw = n_pw = 2     # filter / pooling weight
p = 0               # padding
s = 1               # strride

In [ ]:
def im2col(X, filtersize, padding, strride, layertype) :
    outputsize_h = int((X.shape[2] + 2*padding - filtersize)/strride) + 1  # output size
    outputsize_w = int((X.shape[3] + 2*padding - filtersize)/strride) + 1
    X = np.pad(X,((0,0),(0,0),(padding,padding),(padding,padding)),constant_values=(0,))
    X_col = np.zeros([X.shape[0],X.shape[1], filtersize,filtersize,outputsize_h,outputsize_w])
    for i in range(filtersize) :
        i1 = i; i2 = i1 + strride*outputsize_h
        for j in range(filtersize) :
            j1 = j; j2 = j1 + strride*outputsize_w
            X_col[:,:,i,j,:,:] = X[:,:,i1:i2:strride,j1:j2:strride]
    if layertype == 'conv' :
        X_col = np.transpose(X_col,[0,4,5,1,2,3])
        X_col = np.reshape(X_col,[X.shape[0]*outputsize_h*outputsize_w,X.shape[1]*filtersize*filtersize])
    elif layertype == 'pool' :
        X_col = np.transpose(X_col,[0,1,4,5,2,3])
        X_col = np.reshape(X_col,[X.shape[0]*outputsize_h*outputsize_w*X.shape[1],filtersize*filtersize])
    return X_col

In [ ]:
def col2im(X_col,input_h,input_w ,filtersize, padding, stride) :
    outputsize_h = int((X.shape[2] + 2*padding - filtersize)/stride) + 1  # output size
    outputsize_w = int((X.shape[3] + 2*padding - filtersize)/stride) + 1
    X = np.reshape(X_col,[X_col.shape[0],outputsize_h,outputsize_w,X_col.shape[1],filtersize,filtersize])
    X = np.transpose(X_col,[0,3,4,5,1,2])

    X = np.zeros([X_col.shape[0], X_col.shape[1], input_h+2*padding, input_w+2*padding])
    for i in range(filtersize) :
        i1 = i; i2 = i1 + stride*outputsize_h
        for j in range(filtersize) :
            j1 = j; j2 = j1 + stride * outputsize_w
            X[:,:,i1:i2:stride,j1:j2:stride] += X_col[:,:,i,j,:,:]
    X = X[:,:,padding:input_h+1,padding:input_w+1]
    return X


In [ ]:
class Mul : 
    def forward(self,X,W) :
        self.X = X
        self.W = W
        return np.dot(X,W)
    def backward(self,dY) :
        X = self.X 
        W = self.W
        dX = np.dot(dY,W.T)
        dW = np.dot(X.T,dY)
        return (dX,dW)


In [ ]:
N = 2       # input
n_ic = 2    # input(filter) channel
n_oc = 3    # output channel
n_i = 4     # input(row,col) size
n_f = 3     # filter(row,col) size
p = 0       # padding size
s = 1       # stride size
n_o = int((n_i + 2*p - n_f)/s) + 1  # output size
# X(N, n_ic, n_i, n_i)
X11 = np.array([[1,2,3,0],[0,1,2,3],[3,0,1,2],[2,3,0,1]])
X12 = np.array([[5,6,8,0],[0,6,2,3],[5,2,1,9],[4,3,1,1]])
X1 = np.array([X11,X12])
X21 = np.array([[1,2,1,0],[0,2,2,3],[3,0,9,2],[1,3,0,2]])
X22 = np.array([[5,6,8,0],[2,5,2,3],[2,2,6,9],[4,2,1,4]])
X2 = np.array([X21,X22])
X = np.array([X1,X2])
X = np.pad(X,((0,0),(0,0),(p,p),(p,p)),constant_values=(0,))
show(X,'X')
# W(n_oc, n_ic, n_f, n_f)
W11 = np.array([[2,0,1],[0,1,2],[1,0,2]])
W12 = np.array([[1,2,1],[0,2,0],[1,0,1]])
W1 = np.array([W11,W12])
W21 = np.array([[1,1,1],[1,0,1],[1,2,1]])
W22 = np.array([[1,1,1],[0,1,1],[0,2,2]])
W2 = np.array([W21,W22])
W31 = np.array([[2,1,1],[1,0,0],[1,0,1]])
W32 = np.array([[2,0,1],[2,2,0],[0,2,0]])
W3 = np.array([W31,W32])
W = np.array([W1,W2,W3])
show(W,'W')
# output(N, n_oc, n_o, n_o)


In [ ]:
X = np.random

In [ ]:
class Conv2d :
    def __init__(self,inchannel, outchannel=1, fildtersize=3, padding = 0, stride = 1,) : 
        self.filtersize=fildtersize
        self.padding = padding
        self.stride = stride
        self.outchannel = outchannel
        self.inchannel = inchannel
        self.W = np.sqrt(1./(fildtersize+fildtersize))*np.random.randn(inchannel,outchannel,fildtersize,fildtersize)
        self.mul = Mul()
        
    def forward(self, X):
        self.input_h = X.shape[2]
        self.input_w = X.shape[3]
        self.X_col, oh, ow = im2col(X,self.filtersize,self.padding,self.stride,'conv')
        self.W_col = self.W.reshape([self.outchannel, self.inchannel*self.filtersize*self.filtersize])
        self.W_col = self.W_col.T
        Y = self.mul.forward(self.X_col,self.W_col)
        Y = Y.reshape([X.shape[0],-1,self.outchannel])
        Y = np.transpose(Y,[0,2,1])
        Y = Y.reshape(X.shape[0],self.outchannel,oh,ow)
        return Y
    
    def backword(self,dY) :
        dX,dW = self.mul.backward(dY)
        dX = col2im(dX,self.input_h,self.input_w,self.filtersize,self.padding,self.stride)
        dW = dW.T
        dW.reshape([self.outchannel,self.inchannel,self.filtersize,self.filtersize])
        return dX,dW


In [ ]:
conv2d = Conv2d(n_ic,n_oc,n_f,p,s)
conv2d.forward(X)

In [ ]:
mul = MnistLayers.Mul()
Y = mul.forward(X_col,W_col)
show(Y,'Y')

In [ ]:
Y = Y2img(Y,N,n_oc,n_o,n_o)
show(Y,'Y')

In [ ]:
ReLu = MnistLayers.ReLU()
Yrelu = ReLu.forward(Y)

In [ ]:
show(Yrelu,'Relu')